### Import required libraries

In [ ]:
import pickle
import numpy as np
import seaborn as sns
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

### Define a custom confusion matrix illustrator

In [ ]:
def show_confusion_matrix(y_test, y_pred):
    labels = ["tDCS", "sham", "control"]
    cf_matrix = confusion_matrix(y_test, y_pred)
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap=cmap, cbar=False, yticklabels=labels, xticklabels=labels)

### Load the data frame from the pickle file

In [ ]:
with open("data\DataFrame.pickle", "rb") as file:
    x, y, df= pickle.load(file)

### Split the data frame into two sets: train and test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

### Define some variable for collecting results of the models

In [ ]:
predictions = {}

### *k-nearest neighbors(KNN) Model*

In [ ]:
accuracy_scores = []
for n_index in range(1, len(y_train)-1):
    KNNModel = KNeighborsClassifier(n_neighbors=n_index)
    KNNModel.fit(x_train, y_train)
    y_pred = KNNModel.predict(x_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

n_index=accuracy_scores.index(max(accuracy_scores))+1
KNNModel = KNeighborsClassifier(n_neighbors=n_index)
KNNModel.fit(x_train, y_train)
y_pred = KNNModel.predict(x_test)
predictions["KNN"] = y_pred

#### *Random Forest Classifier Model*

In [ ]:
RFCModel = RandomForestClassifier(n_jobs=-1)
RFCModel.fit(x_train, y_train)
y_pred = RFCModel.predict(x_test)
predictions["Random Forest"] = y_pred

### *Logistic Regression Model*

In [ ]:
LRModel = LogisticRegression(solver="liblinear", max_iter=30000)
LRModel.fit(x_train, y_train)
y_pred = LRModel.predict(x_test)
predictions["Logistic Regression"] = y_pred

### *Support Vector Machine Classifier Model*

In [ ]:
SVCModel = SVC()
SVCModel.fit(x_train, y_train)
y_pred = SVCModel.predict(x_test)
predictions["SVC"] = y_pred

### *XGBoost Model*

In [ ]:
XGBModel = XGBClassifier()
XGBModel.fit(x_train, y_train)
y_pred = XGBModel.predict(x_test)
predictions["XGBoost"] = y_pred

### Ridge

In [ ]:
RidgeModel = RidgeClassifier()
RidgeModel.fit(x_train, y_train)
y_pred = RidgeModel.predict(x_test)
predictions["Ridge"] = y_pred

### *Catboost*

In [ ]:
CBCModel = CatBoostClassifier()
CBCModel.fit(x_train, y_train)
y_pred = CBCModel.predict(x_test)
predictions["CatBoost"] = y_pred

### *Multi-layer Perceptron classifier(MLP classifier)*

In [ ]:
MLPCModel = MLPClassifier()
MLPCModel.fit(x_train, y_train)
y_pred = MLPCModel.predict(x_test)
predictions["MLP"] = y_pred

### Show confusion matrix of each model

In [ ]:
plot_index=1
for response in predictions:
    plt.figure("{0}".format(plot_index), figsize=[3.5, 3.5])
    plt.title(response)
    show_confusion_matrix(y_test, predictions[response])
    plot_index+=1

In [ ]:
for prediction in predictions:
    print("{0} :".format(prediction))
    print(classification_report(y_test, predictions[prediction], zero_division=0))

In [ ]:
print(x_train.index)
print(x_test.index)

In [ ]:
accuracy_scores={}
for prediction in predictions:
    accuracy_scores[prediction]=accuracy_score(y_test, predictions[prediction])

In [ ]:
plt.figure(figsize=[10, 5])
sns.barplot(x=list(accuracy_scores.keys()), y=list(accuracy_scores.values()), palette="crest")
plt.show()